In [ ]:
!pip install flask
!pip install pyngrok



In [2]:
!pip install gspread google-auth-oauthlib google-auth-httplib2
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.5 MB/s eta 0:00:00


In [3]:
from flask import send_from_directory

In [4]:
!pip install pydub
!pip install transformers==4.32.0
!pip install -U denoiser
!pip install noisereduce
!pip install torchaudio
!pip install librosa
!pip install pyannote.audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 62.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully uninstalled tokenizers-0.20.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.15 requires transformers>=4.33.1, but you have transformers 4.32.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 2.2 MB/s eta 0:00:00
  Preparing me

In [5]:
import gspread
from google.oauth2.service_account import Credentials
import google.generativeai as genai
import os
import json
import os
import logging
import time
import threading
from flask import Flask, request, jsonify
from pyngrok import ngrok
from pydub import AudioSegment
import torch
import torchaudio
import urllib.request
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import glob
import os
import base64
import numpy as np
from pydub import AudioSegment
from pyannote.audio import Model, Audio
from pyannote.audio.pipelines import VoiceActivityDetection
from pyannote.core import Segment
from transformers import pipeline
from sklearn.cluster import AgglomerativeClustering
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
import torch
from kaggle_secrets import UserSecretsClient 

/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [6]:
try:
    user_secrets = UserSecretsClient()
    
    # Get the Google Sheets/GCP key
    gcp_sa_key_string = user_secrets.get_secret("GCP_SA_KEY")
    os.environ['GCP_SA_KEY'] = gcp_sa_key_string
    print("Successfully loaded GCP_SA_KEY from Kaggle Secrets into environment.")
    
    # Get the Gemini API key
    gemini_api_key = user_secrets.get_secret("GEMINI_API_KEY")
    os.environ['GEMINI_API_KEY'] = gemini_api_key
    print("Successfully loaded GEMINI_API_KEY from Kaggle Secrets into environment.")

except Exception as e:
    print(f"CRITICAL ERROR: Could not load secrets from Kaggle. Error: {e}")
    # Set to empty strings so downstream functions fail gracefully
    os.environ['GCP_SA_KEY'] = ""
    os.environ['GEMINI_API_KEY'] = ""

Successfully loaded GCP_SA_KEY from Kaggle Secrets into environment.
Successfully loaded GEMINI_API_KEY from Kaggle Secrets into environment.


In [7]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Define directories
INPUT_FOLDER = '/kaggle/working/input_wav_files'
OUTPUT_FOLDER = '/kaggle/working/output_text_files'
os.makedirs(INPUT_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Initialize Flask app
app = Flask(__name__)

In [ ]:
# Set ngrok authtoken (replace with your actual ngrok authtoken)
NGROK_AUTHTOKEN = ''  # Set this securely, e.g., via environment variables
ngrok.set_auth_token(NGROK_AUTHTOKEN)

In [9]:
def update_spreadsheet(spreadsheet_name, worksheet_name, file_name, whisper_output, google_output):
    """
    Appends a new row to the specified Google Sheet with the transcription data.
    """
    logging.info("Attempting to update Google Sheet...")
    try:
        # Load credentials from Kaggle Secrets
        sa_key_json_string = os.environ.get('GCP_SA_KEY')
        if not sa_key_json_string:
            logging.error("Kaggle Secret 'GCP_SA_KEY' not found.")
            return False
            
        creds_dict = json.loads(sa_key_json_string)
        
        scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
        creds = Credentials.from_service_account_info(creds_dict, scopes=scopes)
        client = gspread.authorize(creds)

        # Open the spreadsheet and the specific worksheet
        spreadsheet = client.open(spreadsheet_name)
        worksheet = spreadsheet.worksheet(worksheet_name)
        
        # Ensure header row exists
        header = ["file_name", "whisper_output", "structured_text"]
        try:
            # Check if the first row matches the header
            if worksheet.row_values(1) != header:
                worksheet.insert_row(header, 1)
        except gspread.exceptions.APIError: # Likely an empty sheet
             worksheet.insert_row(header, 1)


        # Prepare the row to be added
        new_row = [file_name, whisper_output, google_output]
        
        # Append the new row to the worksheet
        worksheet.append_row(new_row)
        logging.info(f"Successfully updated spreadsheet for file: {file_name}")
        return True
    except Exception as e:
        logging.error(f"Error updating spreadsheet: {e}")
        return False


In [10]:
# Convert input audio to WAV format
def convert_to_wav(input_path, output_path):
    if isinstance(input_path, bytes):  # If input is Base64
        decoded = base64.b64decode(input_path)
        with open("temp_input.mp3", "wb") as f:
            f.write(decoded)
        audio = AudioSegment.from_file("temp_input.mp3")
    else:  # If input is a file path
        audio = AudioSegment.from_file(input_path)

    audio.export(output_path, format="wav")
    return output_path

In [11]:
# Set up device
device = "cuda" if torch.cuda.is_available() else "cpu"
device_index = 0 if device == "cuda" else -1

# Load ASR pipeline (Bangla whisper model for example)
asr_model_name = "sakibzaman/AIMSCRIBE_WHISPER_BN"
asr_pipeline = pipeline(task="automatic-speech-recognition", model=asr_model_name, tokenizer=asr_model_name, chunk_length_s=8, device=device_index)
asr_pipeline.model.config.forced_decoder_ids = asr_pipeline.tokenizer.get_decoder_prompt_ids(language="bn", task="transcribe")

# Load VAD model


/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location=map

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

In [ ]:
vad_model = Model.from_pretrained("pyannote/segmentation", use_auth_token="")
vad_pipeline = VoiceActivityDetection(segmentation=vad_model, device=torch.device(device))
# Load speaker embedding model
embedding_model = PretrainedSpeakerEmbedding("speechbrain/spkrec-ecapa-voxceleb", device=torch.device(device))
audio_processor = Audio()

pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/318 [00:00<?, ?B/s]

Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.0. Bad things might happen unless you revert torch to 1.x.


hyperparams.yaml: 0.00B [00:00, ?B/s]

/opt/conda/lib/python3.10/site-packages/speechbrain/utils/autocast.py:188: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt: 0.00B [00:00, ?B/s]

/opt/conda/lib/python3.10/site-packages/speechbrain/utils/checkpoints.py:200: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

In [13]:
def process_audio(input_path, num_speakers=2):
    """
    This is the definitive, efficient, and correct function.
    It uses the "Organize -> Transcribe" strategy and now includes a
    robust check to ensure all audio chunks are mono.
    """
    print(f"--- Starting Diarization for: {os.path.basename(input_path)} ---")

    # =================================================================================
    # SETUP
    # =================================================================================
    file_name = os.path.basename(input_path).split('.')[0]
    parent_folder = os.path.dirname(input_path)
    output_text_path = os.path.join(OUTPUT_FOLDER, f"{file_name}.txt")
    wav_path = convert_to_wav(input_path, "temp_processing_audio.wav")
    audio_duration = AudioSegment.from_wav(wav_path).duration_seconds

    # =================================================================================
    # STEP 1: Voice Activity Detection (VAD)
    # =================================================================================
    vad_pipeline.instantiate({
        "onset": 0.5, "offset": 0.6, "min_duration_on": 0.1, "min_duration_off": 0.1
    })
    print("Step 1: Finding all speech segments using VAD...")
    vad_result = vad_pipeline(wav_path)
    speech_timeline = vad_result.get_timeline().support()

    # =================================================================================
    # STEP 2: Speaker Embedding Extraction
    # =================================================================================
    print("Step 2: Creating a 'voiceprint' for each speech segment...")
    embeddings = []
    segments_for_clustering = []

    for segment in speech_timeline:
        start = segment.start
        end = segment.end

        if (end - start) < 0.1:
            continue

        waveform, sample_rate = torchaudio.load(wav_path, frame_offset=int(start * 16000), num_frames=int((end - start) * 16000))

        # ===================================================================
        # THE FIX IS HERE: Force the tensor to be mono BEFORE reshaping.
        # ===================================================================
        # If the tensor has more than one channel (e.g., stereo with shape [2, 16000]),
        # we average the channels to create a mono tensor (shape [1, 16000]).
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)
        # ===================================================================

        # This is the CRITICAL TENSOR SHAPING LOGIC.
        if waveform.dim() == 1:
            waveform = waveform.unsqueeze(0)
        waveform_batch = waveform.unsqueeze(0)

        embedding = embedding_model(waveform_batch.to(device))

        final_embedding = None
        if isinstance(embedding, torch.Tensor):
            # If the model returns a tensor, convert it to a numpy array.
            final_embedding = embedding.squeeze().cpu().numpy()
        elif isinstance(embedding, np.ndarray):
            # If the model already returned a numpy array, just use it.
            final_embedding = np.squeeze(embedding)
        else:
            raise TypeError(f"The embedding model returned an unexpected type: {type(embedding)}")

        embeddings.append(final_embedding)
        # ===================================================================

        segments_for_clustering.append({'start': start, 'end': end})

    if not embeddings:
        print("Error: No speech detected in the audio file. Cannot proceed.")
        os.remove(wav_path)
        return

    # =================================================================================
    # STEP 3: Speaker Clustering
    # =================================================================================
    print("Step 3: Grouping voiceprints to identify speakers...")
    clustering = AgglomerativeClustering(n_clusters=num_speakers).fit(np.array(embeddings))
    labels = clustering.labels_
    for i, segment in enumerate(segments_for_clustering):
        segment['speaker'] = f"Speaker {labels[i] + 1}"

    # =================================================================================
    # STEP 4: Merging into Speaker Turns
    # =================================================================================
    print("Step 4: Reconstructing conversation turns from segments...")
    speaker_turns = []
    if segments_for_clustering:
        current_turn = segments_for_clustering[0].copy()
        for i in range(1, len(segments_for_clustering)):
            segment = segments_for_clustering[i]
            if segment['speaker'] == current_turn['speaker'] and (segment['start'] - current_turn['end']) < 1.5:
                current_turn['end'] = segment['end']
            else:
                speaker_turns.append(current_turn)
                current_turn = segment.copy()
        speaker_turns.append(current_turn)

    # =================================================================================
    # STEP 5: Transcription
    # =================================================================================
    print("Step 5: Transcribing each speaker turn...")
    full_transcription = []
    audio_file = AudioSegment.from_wav(wav_path)

    for i, turn in enumerate(speaker_turns):
        start_ms = int(turn['start'] * 1000)
        end_ms = int(turn['end'] * 1000)

        turn_audio_segment = audio_file[start_ms:end_ms]
        temp_turn_path = f"temp_turn_{i}.wav"
        turn_audio_segment.export(temp_turn_path, format="wav")

        transcription_result = asr_pipeline(temp_turn_path)["text"]
        os.remove(temp_turn_path)

        formatted_line = f"[{turn['speaker']} | {turn['start']:.2f}-{turn['end']:.2f}] {transcription_result.strip()}"
        full_transcription.append(formatted_line)

    # =================================================================================
    # FINAL OUTPUT AND CLEANUP
    # =================================================================================
    print("\n--- FINAL DIARIZED TRANSCRIPT ---")
    print("\n".join(full_transcription))

    print(f"\nTranscription saved to: {output_text_path}")
    with open(output_text_path, "w", encoding="utf-8") as f:
        f.write("\n".join(full_transcription))

    os.remove(wav_path)

    return output_text_path


In [15]:
# === PASTE THIS ENTIRE FUNCTION INTO YOUR KAGGLE NOTEBOOK ===

@app.route('/upload/v1/<fileName>', methods=['POST'], endpoint='upload_file')
def upload_file(fileName):
    try:
        # --- 1. File Validation and Saving ---
        if 'file' not in request.files:
            return jsonify({'error': 'No file provided'}), 400
        file = request.files['file']
        if file.filename == '' or not file.filename.endswith('.wav'):
            return jsonify({'error': 'Invalid file format. Please upload a .wav file'}), 400
        if not fileName.endswith('.wav'):
            return jsonify({'error': 'fileName must end with .wav'}), 400

        wav_path = os.path.join(INPUT_FOLDER, fileName)
        file.save(wav_path)
        logging.info(f'Saved file: {wav_path}')

        # --- 2. Run Whisper/Pyannote Diarization ---
        logging.info("Running Process 1: Whisper + pyannote diarization...")
        transcript_file_path = process_audio(wav_path) 
        
        whisper_diarized_output = ""
        if transcript_file_path and os.path.exists(transcript_file_path):
             with open(transcript_file_path, 'r', encoding='utf-8') as f:
                whisper_diarized_output = f.read()
        logging.info("Process 1 complete.")


        # --- 4. Update the Google Spreadsheet ---
        logging.info("Updating Google Sheet with both diarized outputs...")
        SPREADSHEET_NAME = "My Transcription Project"
        WORKSHEET_NAME = "Sheet1"
        update_spreadsheet(
            spreadsheet_name=SPREADSHEET_NAME,
            worksheet_name=WORKSHEET_NAME,
            file_name=fileName,
            whisper_output=whisper_diarized_output
        )

        # --- 5. Clean Up ONLY the Original Audio File ---
        os.remove(wav_path)
        
        # FIX #1: The transcript file path is now correctly added to the response.
        # Your local client is looking for the key 'output_file'.
        transcript_filename = os.path.basename(transcript_file_path) if transcript_file_path else None

        # FIX #2: The premature deletion of the transcript file is REMOVED.
        # The file will now persist so the /download endpoint can serve it.
        # os.remove(transcript_file_path) # <-- THIS LINE IS DELETED

        # --- 6. Send the Final, Correct Response ---
        return jsonify({
            'message': 'File processed by both systems and spreadsheet updated.',
            'file_name': fileName,
            'output_file': transcript_filename, # <-- THIS KEY IS NOW PRESENT
            'whisper_plus_pyannote_output': whisper_diarized_output
        }), 200

    except Exception as e:
        logging.error(f'Error processing file {fileName}: {e}', exc_info=True)
        return jsonify({'error': str(e)}), 500

In [16]:
@app.route('/download/v1/<transcript_filename>', methods=['GET'])
def download_transcript(transcript_filename):
    """
    This endpoint allows the client to download a generated transcription file.
    It looks for the file inside the OUTPUT_FOLDER.
    """
    logging.info(f"Received download request for: {transcript_filename}")
    try:
        # Use Flask's secure send_from_directory function.
        # It safely handles file paths and prevents directory traversal attacks.
        # as_attachment=True tells the client to treat this as a file to be saved.
        return send_from_directory(
            directory=OUTPUT_FOLDER, 
            path=transcript_filename, 
            as_attachment=True
        )
    except FileNotFoundError:
        # If the requested file doesn't exist, return a proper 404 error.
        logging.error(f"Download request failed: File not found at {os.path.join(OUTPUT_FOLDER, transcript_filename)}")
        return jsonify({"error": "File not found"}), 404
    except Exception as e:
        logging.error(f"An unexpected error occurred during download: {e}")
        return jsonify({"error": "An internal server error occurred"}),500

In [17]:
# Start Flask app in a separate thread
def run_flask():
    app.run(host='0.0.0.0', port=6000)

# Start ngrok tunnel
def start_ngrok():
    public_url = ngrok.connect(6000, domain="internal-polliwog-eagerly.ngrok-free.app")
    logging.info(f'Ngrok tunnel established at: {public_url}')
    print(f'Ngrok public URL: {public_url}')
    return public_url

In [ ]:
if __name__ == '__main__':
    # Install required packages
    os.system('pip install pydub')
    os.system('pip install transformers==4.32.0')
    os.system('pip install -U denoiser')
    os.system('pip install noisereduce')
    os.system('pip install torchaudio')
    os.system('pip install librosa')
    os.system('pip install pyannote.audio')
    os.system('pip install flask')
    os.system('pip install pyngrok')

    # Run Flask and ngrok
    flask_thread = threading.Thread(target=run_flask)
    flask_thread.daemon = True
    flask_thread.start()

    ngrok_url = start_ngrok()

    # Keep the script running
    try:
        while True:
            time.sleep(10)
    except KeyboardInterrupt:
        ngrok.disconnect(ngrok_url)
        logging.info('Ngrok tunnel disconnected')
        exit(0)

  Using cached omegaconf-1.4.1-py3-none-any.whl.metadata (2.9 kB)
Using cached omegaconf-1.4.1-py3-none-any.whl (14 kB)
  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 2.3.0
    Uninstalling omegaconf-2.3.0:
      Successfully uninstalled omegaconf-2.3.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyannote-audio 3.3.2 requires omegaconf<3.0,>=2.1, but you have omegaconf 1.4.1 which is incompatible.


  Using cached omegaconf-2.3.0-py3-none-any.whl.metadata (3.9 kB)
Using cached omegaconf-2.3.0-py3-none-any.whl (79 kB)
  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 1.4.1
    Uninstalling omegaconf-1.4.1:
      Successfully uninstalled omegaconf-1.4.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hydra-core 0.11.3 requires omegaconf<1.5,>=1.4, but you have omegaconf 2.3.0 which is incompatible.


 * Serving Flask app '__main__'
 * Debug mode: off
Ngrok public URL: NgrokTunnel: "https://internal-polliwog-eagerly.ngrok-free.app" -> "http://localhost:6000"
--- Starting Diarization for: 4_Sakib_UIUHUB_17_16_17_22_2025_06_22.wav ---
Step 1: Finding all speech segments using VAD...


/opt/conda/lib/python3.10/site-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(


Step 2: Creating a 'voiceprint' for each speech segment...
Step 3: Grouping voiceprints to identify speakers...
Step 4: Reconstructing conversation turns from segments...
Step 5: Transcribing each speaker turn...


/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentiall


--- FINAL DIARIZED TRANSCRIPT ---
[Speaker 2 | 2.12-7.88] দেখেন আমি অনেক দূর থেকে কথা বলছি কিন্তু হ্যাঁ তোমার হচ্ছে পরে হচ্ছে ইয়াপ ভালো আছে
[Speaker 1 | 9.90-13.83] আমি কিন্তু সাধ্য ঝুঁকি নাই আমি এরকম করি না
[Speaker 2 | 31.33-61.54] ডাক্তার রাফির কী অবস্ আপনার জিনিস পাতিয়ে রোগী পেছে ওদিকে হচ্ছ্কাসের সাথে দিয়ে নিয়ে যে বলতে হবে ভাই তুমি তো অতসী
[Speaker 1 | 61.68-66.27] খাইতে চাও তো তোমার জিনিসই দিবা না এবার কোম্পিট না করলে তো হচ্ছে শুভ হবার জিও হবে
[Speaker 2 | 67.43-70.89] আমাকে বার্তা নির্বাচন করতে দাও অথবা বার্তা কম্পিট হলেই অতসী পাবে
[Speaker 1 | 71.51-72.91] ওহ বান্ধবী অনুযায়ী
[Speaker 2 | 75.01-79.48] কী অবস্থা এখান থেকে কি সাউন্ড থেকে কি টেক্সটার থাকার কারণ এখন
[Speaker 1 | 79.71-81.59] করতেছি তো
[Speaker 2 | 85.35-96.24] আপনাকে ডেভিড সিটি চলতেছে ভাই হ্যাঁ ওকে পালাতে হতে পারি ও ডক্টর খান আমার এই ব্যাংক হবে আর কি মানে এই ক্রমের বাধাটা একটু খান হবে
[Speaker 2 | 99.91-103.12] স্বামীকে রেগুলি না খুব মনে লাগবে
[Speaker 1 | 105.48-109.30] বা এটা সাদেকে নিতে বলেন আপনাদের
[Speaker 2 | 